In [5]:
import requests
import pandas as pd
import json
import pymysql

ImportError: attempted relative import with no known parent package

In [2]:
import logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s]: %(message)s',
    datefmt='%d/%m/%Y %H:%M:%S'
)

In [7]:
host = "localhost"
user = "root"
db_name = 'dapp_analysis_rearrange'
db = pymysql.connect(host = host, user = user, db = db_name)
cursor = db.cursor()

In [8]:
df = pd.read_csv("../data/for_crawling.csv")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4707 entries, 0 to 4706
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4707 non-null   int64 
 1   tx_hash     4707 non-null   object
dtypes: int64(1), object(1)
memory usage: 73.7+ KB


In [10]:
import time
# sess = requests.Session()

headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

def get_graph(txHash):
    url = "https://bloxy.info/tx_graph_expanded_results/" + txHash
    
    try:
        resp = requests.get(url, headers = headers)
        if resp.status_code == 200:
            return resp.json()
        
        logging.error("[Other status code]\n" + \
                f"Statuc code {resp.status_code} at {url}\n" + \
                f"{resp.text}")
        
    except Exception as e:
        logging.error("[Unexpected error]\n" + \
                f"{e}\n{url}\n")
        
    # logging.info("==> Sleep 3 minutes")
    # time.sleep(180)
    # logging.info("Come back, request again")
    
    q = "INSERT INTO error_tx_trace(tx_hash) VALUES (%s)"
    cursor.execute(q, (txHash,))
    db.commit()
    return None

query = "INSERT INTO tx_trace(tx_hash, trace_count, trace_graph) VALUES (%s, %s, %s)"
           
isExist = True
for index, row in df.iterrows():
    txHash = row['tx_hash']
    
    trace_graph = get_graph(txHash)
    if trace_graph:
        trace_len = len(trace_graph)
            
        params = (txHash, trace_len, json.dumps(trace_graph))
        cursor.execute(query, params)
        db.commit()
    time.sleep(1)

07/01/2024 13:35:33 [DEBUG]: Starting new HTTPS connection (1): bloxy.info:443


07/01/2024 13:35:38 [DEBUG]: https://bloxy.info:443 "GET /tx_graph_expanded_results/0x18097d9f175422ca134e914d3228b6d2481dca311e36bbf2a700406349901e5d HTTP/1.1" 200 None
07/01/2024 13:35:39 [DEBUG]: Starting new HTTPS connection (1): bloxy.info:443
07/01/2024 13:35:40 [DEBUG]: https://bloxy.info:443 "GET /tx_graph_expanded_results/0xdd675dc28f33db89b4e1a18215d2f6faf111b23b32580ac4bb43377b4f2e4827 HTTP/1.1" 200 None
07/01/2024 13:35:41 [DEBUG]: Starting new HTTPS connection (1): bloxy.info:443
07/01/2024 13:35:44 [DEBUG]: https://bloxy.info:443 "GET /tx_graph_expanded_results/0x5a8721a6dbdf2262d1a41ea6af81afee7657be219649c784e03664f2ee115d35 HTTP/1.1" 200 None
07/01/2024 13:35:45 [DEBUG]: Starting new HTTPS connection (1): bloxy.info:443
07/01/2024 13:35:48 [DEBUG]: https://bloxy.info:443 "GET /tx_graph_expanded_results/0xed3621e403ed822da520084aa102da182ad30f349ae4898369067052f868d035 HTTP/1.1" 200 None
07/01/2024 13:35:49 [DEBUG]: Starting new HTTPS connection (1): bloxy.info:443
07/0

KeyboardInterrupt: 

In [ ]:
import os
os.system("shutdown /s /t 0")